In [30]:
import os
import sys
sys.path.insert(0, "../src")
import tensorflow as tf

import gym
import baselines
from baselines import deepq

import normalized_env

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
import math
import numpy as np

def modify_env(env):

    # hack to give us control over the initial state
    import types

    def new_timelimt_step(self, action):
        assert self._episode_started_at is not None, "Cannot call env.step() before calling reset()"
        observation, reward, done, info = self.env.step(action)
        self._elapsed_steps += 1

        if self._past_limit():
            if self.metadata.get('semantics.autoreset'):
                _ = self.reset() # automatically reset the env
            done = True 

        return observation, reward, done, info

    def new_step(self, discrete_action):
        disc_to_cont = {
            0: [-1.0],
            1: [-0.5],
            2: [0.0],
            3: [0.5],
            4: [1.0]
        }

        action = disc_to_cont[int(discrete_action[0])]
        # hacky way to map continuous action to discrete value

        position = self.state[0]
        velocity = self.state[1]
        force = min(max(action[0], -1.0), 1.0)

        velocity += force*self.power -0.0025 * math.cos(3*position)
        if (velocity > self.max_speed): velocity = self.max_speed
        if (velocity < -self.max_speed): velocity = -self.max_speed
        position += velocity
        if (position > self.max_position): position = self.max_position
        if (position < self.min_position): position = self.min_position
        if (position==self.min_position and velocity<0): velocity = 0

        done = bool(position >= self.goal_position)

        reward = 0
        if done:
            reward = 100.0
        reward-= math.pow(action[0],2)*0.1

        self.state = np.array([position, velocity])

        return self.state, reward, done, {}
    # set max time step
    
    env._max_episode_steps = 999
    
    #env.reset = types.MethodType(new_timelimit_reset, env)
    env.env.step = types.MethodType(new_step, env.env)
    env = normalized_env.make_normalized_env(env)
    return env


In [27]:
env_id = "MountainCarContinuous-v0"
env = gym.envs.make(env_id)
env = modify_env(env)
env._max_episode_steps

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
True action space: [-1.], [1.]
True state space: [-1.2  -0.07], [0.6  0.07]
Filtered action space: [-1.], [1.]
Filtered state space: [-1. -1.], [1. 1.]


999

In [31]:
def main():
    env_id = "MountainCarContinuous-v0"
    env = gym.envs.make(env_id)
    env = modify_env(env)
    
    model = deepq.models.mlp([200, 200], layer_norm=True)
    act = deepq.learn(
        env,
        n_action=5,
        q_func=model,
        lr=1e-3,
        max_timesteps=20000,
        buffer_size=100000,
        exploration_fraction=0.2,
        exploration_final_eps=0.05,
        print_freq=5,
        param_noise=True,
        target_network_update_freq=1000,
        gamma=0.99,
        prioritized_replay=True,
        prioritized_replay_alpha=0.6,
        prioritized_replay_beta0=0.9,
        prioritized_replay_beta_iters=100000,
        prioritized_replay_eps=1e-6
    )
    act.save("mountaincar_model.pkl")


In [ ]:
with tf.Graph().as_default():
    main()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
True action space: [-1.], [1.]
True state space: [-1.2  -0.07], [0.6  0.07]
Filtered action space: [-1.], [1.]
Filtered state space: [-1. -1.], [1. 1.]
============== training starts ==============


/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
